In [ ]:
#### Some of the implementation of the full transformer architecture was adapted from the imperial NLP course
# https://github.com/ImperialNLP/NLPLabs-2022/tree/main/transformers_code_answers

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple, defaultdict
import math
import yfinance as yf
import os
from pytorch_lightning import seed_everything
import random
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


In [ ]:
np.random.seed(1233)
torch.manual_seed(1233)
np.random.seed(1233)
seed_everything(1233, workers=True)
os.environ["CUBLAS_WORKSPACE_CONFIG"]=":4096:2"

In [ ]:
device = ("cuda:0" if torch.cuda.is_available else "cpu")

#### model

In [ ]:
class AbsolutePositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.3, max_seq_len=200, device="cpu"):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_seq_len, d_model).to(device)
        pos = torch.arange(0, max_seq_len).unsqueeze(1).float()

        two_i = torch.arange(0, d_model, step=2).float()
        div_term = torch.pow(10000, (two_i/torch.Tensor([d_model]))).float()
        pe[:, 0::2] = torch.sin(pos/div_term)
        pe[:, 1::2] = torch.cos(pos/div_term)

        pe = pe.unsqueeze(0)

        self.register_buffer("pe", pe)

    def forward(self, x):
        # shape(x) = [B x seq_len x D]
        pe = self.pe[:, :x.shape[1]].detach()
        x = x.add(pe)
        # shape(x) = [B x seq_len x D]
        return self.dropout(x)

In [ ]:
class Time2VecPositonalEncoding(nn.Module):
    def __init__(self,input_dim, output_dim, activation="sine"):
        super().__init__()
        self.linear_layer = nn.Linear(input_dim, 1)
        self.periodic_layer = nn.Linear(input_dim, output_dim - 1)
        self.activation = activation

    def forward(self, x):

        if self.activation == "sine":
            periodic_out = torch.sin(self.periodic_layer(x))
        elif self.activation == "cos":
            periodic_out = torch.cos(self.periodic_layer(x))

        
        original_out = self.linear_layer(x)

        out = torch.cat([periodic_out, original_out], 2)
        
        return out

In [ ]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, config):
        super().__init__()

        # print(config)
        dropout = config["dropout"]
        device = config["device"]
        max_seq_len = config["max_seq_len"]
        self.dropout = nn.Dropout(p=dropout)

        input_dim, d_model = config["input_dim"], config["d_model"]
        self.inputProjection = nn.Linear(input_dim, d_model)
        self.targetProjection = nn.Linear(input_dim, d_model)


        n_head, num_enc_layers, num_dec_layers = config["n_head"], config["num_enc_layers"], config["num_dec_layers"] 
        self.transformer = nn.Transformer(d_model, n_head, num_enc_layers, num_dec_layers, 
            dim_feedforward= 4 * d_model, dropout=dropout, batch_first=True, device=device)
        
        self.encoderLayer = nn.TransformerEncoderLayer(d_model, n_head, dim_feedforward=4 * d_model, dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(self.encoderLayer, num_enc_layers)

        steps_ahead = config["steps_ahead"]
        self.linear = nn.Linear(d_model, 1)
        self.decoder = nn.Linear(d_model * steps_ahead, steps_ahead)
        self.d_model = d_model
        self.use_encoder_only = config["use_encoder_only"]
        self.useInputProjection = config["use_input_projections"]

        self.isClassification = config["is_classification"] 
         

        if config["use_absolute_enc"]:
            self.inputPosEncoding = AbsolutePositionalEncoding(d_model, dropout, max_seq_len, device)
            self.outputPosEncoding = AbsolutePositionalEncoding(d_model, dropout, max_seq_len, device)
        else:
            time2vecActivation = config["activation"]
            self.inputPosEncoding = Time2VecPositonalEncoding(input_dim, d_model, time2vecActivation)
            self.outputPosEncoding = Time2VecPositonalEncoding(input_dim, d_model, time2vecActivation)
    
    
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        
    def create_mask(self, size):
        return torch.triu(torch.ones(size, size) * float('-inf'), diagonal=1)
    
    def forward(self, src, trg):
        src_mask = self.create_mask(src.shape[1])
        src_mask = src_mask.type_as(src)
        trg_mask = self.create_mask(trg.shape[1])
        trg_mask = trg_mask.type_as(trg)
        
        if self.useInputProjection:
            src = self.inputProjection(src)
            trg = self.targetProjection(trg)

        if self.use_encoder_only:
            src = self.inputPosEncoding(src)
            out = self.encoder(src, src_mask)
            out = self.linear(out)
            if self.isClassification:
                out = F.log_softmax(out)
            return out
        else:
            src = self.inputPosEncoding(src)
            trg = self.outputPosEncoding(trg) 
 
            out = self.transformer(src, trg, tgt_mask=trg_mask)
            out = self.linear(out)
            # out = self.decoder(out.flatten(start_dim=1))
            if self.isClassification:
                out = F.log_softmax(out)
            return out


In [ ]:
def r2_loss(output, target):
    target_mean = torch.mean(target)
    ss_tot = torch.sum((target - target_mean) ** 2)
    ss_res = torch.sum((target - output) ** 2)
    r2 = 1 - ss_res / ss_tot
    return r2

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def get_prediction(t):
    return [arr.view(-1) for arr in t]

In [ ]:
class TransformerTrainer(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.save_hyperparameters()

        config["device"] = self.hparams.config['device']
        self.model = TimeSeriesTransformer(config)

        self.d_model= config["d_model"]
        self.warmup_steps = config["warmup_steps"]
        self.lr = config["lr"]
        self.device_ = self.hparams.config['device']
        self.config = config
        self.training_len = config["training_len"]
        self.sampling = config["sampling"]
        self.forecast_window = config["steps_ahead"]
        self.k = config["k"]
        self.threshold = config["threshold"]

        if config["loss"] == 'mse':
            self.criterion = nn.MSELoss(reduction='sum')
        else:
            self.criterion = r2_loss


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

    def change_lr_in_optimizer(self):
        min_arg1 = math.sqrt(self.global_step)
        min_arg2 = self.global_step * (self.warmup_steps**-1.5)
        lr = math.sqrt(self.d_model) * min(min_arg1, min_arg2)
        self.trainer.lightning_optimizers[0].param_groups[0]['lr'] = lr
    
    def prob_get_true_val(self, p):
        return random.random() < p

    def n_step_forward(self, src, trg):
        sampled_input = src
        all_preds = [] 

        # src.shape: [64, 60, 1]
        # tgt_in.shape: [64, 30, 1]
        # tgt_out: [64, 30]

        for i in range(self.forecast_window):
           
            pred = self.model(sampled_input, trg[:, i, :].unsqueeze(-1))
            ## pred shape: [B * 1 * 1]

            if all_preds == []:
                all_preds = pred
            else:
                all_preds = torch.cat((all_preds, pred), dim=1)

            p = self.k / (self.k + math.exp(self.current_epoch/self.k))

            prob_true_val = True
            if self.sampling and i > self.threshold:
                prob_true_val = self.prob_get_true_val(p)
            
            if prob_true_val and i > self.threshold:
                sampled_input = torch.cat((sampled_input[:, 1:, :].detach(), trg[:, i, :].unsqueeze(-1).detach()), dim=1)
            else:
                sampled_input = torch.cat((sampled_input[:, 1:, :].detach(), pred.detach()), dim=1)
            
        return all_preds

    def training_step(self, batch, batch_idx):
        src, trg_in, targets = batch

        y_hat = self.n_step_forward(src, trg_in)
        y_hat = y_hat.squeeze(-1)
        y = targets

        loss = self.criterion(y_hat, y)

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        pearson_corr = self.calc_pearson_coeff(y_hat, y)

        self.log("train_pearson_coef", pearson_corr ,on_step=True, on_epoch=True, prog_bar=True, logger=True)

        self.change_lr_in_optimizer()
        return loss
    
    def training_epoch_end(self, outputs): 
        avg_loss = torch.stack(
            [x["loss"].detach() for x in outputs]).mean()

        if self.current_epoch % 5 == 0:
            self.print_predictions(train_loader_unshuffled, train_output_img_dir+"train", "Training")
        
        self.log_dict({"train_loss_epoch": avg_loss, "step":self.current_epoch})

    
    def print_predictions(self, dataloader, file_prefix, title="Validation"):
        # print predictiion of first 1024 + 100 data points
        with torch.no_grad():
            all_predictions = []
            init_train_data = []
            all_targets = []
            for step, trng_data in enumerate(dataloader):
                if step == 1:
                    break

                src, _, targets = trng_data
                all_targets.append(targets[0].detach().cpu())

                if step == 0:
                    init_train_data = src[0, :, 0].reshape(-1)

                prediction = self(trng_data)
                all_predictions.append(prediction[0].detach().cpu())

          
            all_predictions = flatten(get_prediction(all_predictions))
            all_predictions = np.array(all_predictions)
            all_targets = flatten(get_prediction(all_targets))
            all_targets = np.array(all_targets)

            # print(all_predictions.shape)
            # print(all_targets.shape)
            # print(init_train_data.shape)

            # inverse transform
            init_train_data = train_targets_scaler.inverse_transform(init_train_data.reshape(-1, 1)).reshape(-1)
            all_predictions = train_targets_scaler.inverse_transform(all_predictions.reshape(-1, 1)).reshape(-1)
            all_targets = train_targets_scaler.inverse_transform(all_targets.reshape(-1, 1)).reshape(-1)

            end_plot_idx = self.training_len + len(all_predictions)
            plt.figure(figsize=(8, 6))
            plt.plot(init_train_data, label='trailing')
            plt.plot(np.arange(self.training_len, end_plot_idx), all_predictions, label="predicted")
            plt.plot(np.arange(self.training_len, end_plot_idx), all_targets, label="actual")
            plt.title(f"{title} prediction for epoch {self.current_epoch}")
            plt.legend()
            plt.grid()

            if not os.path.exists(file_prefix):
                os.makedirs(file_prefix)

            plt.savefig(f"{file_prefix}/Next_{len(all_predictions)}_preds_Epoch_{self.current_epoch}.jpg", bbox_inches="tight")
            
            plt.close()
            # plt.show()
    
    def validation_step(self, batch, batch_idx):
        src, trg_in, targets = batch

        y_hat = self.n_step_forward(src, trg_in)
        y_hat = y_hat.squeeze(-1)
        y = targets

        loss = self.criterion(y_hat, y)
        
        self.log("valid_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        pearson_corr = self.calc_pearson_coeff(y_hat, y)

        self.log("val_pearson_coef", pearson_corr ,on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss 
    

    def calc_pearson_coeff(self, y_pred, y):

        predicted = y_pred
        targets = y
        vy_pred = predicted - torch.mean(predicted)
        vy = targets - torch.mean(targets)
        denom = torch.sum(vy_pred ** 2) * torch.sum(vy ** 2)

        corr = torch.sum(vy_pred * vy) / torch.sqrt(denom)

        return corr
    
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack(
            [x.detach() for x in outputs]).mean()

        self.log("ptl/val_loss", avg_loss)

        if self.current_epoch % 5 == 0:
            self.print_predictions(val_loader, train_output_img_dir+"val")
        
        self.log_dict({"valid_loss_epoch": avg_loss, "step":self.current_epoch})

        
    def forward(self, x):
        src, trg_in, _ = x
        src = src.to(self._device)
        trg_in = trg_in.to(self._device)
        return self.n_step_forward(src, trg_in)
    

#### Create Sequence Dataset

In [ ]:
f_multiplier = 1_000_000
f_multiplier = 1
class FeaturesDataset(Dataset):
    def __init__(self, features, targets, training_len, forecast_len):
        super().__init__()
        self.features = features
        self.targets = targets
        self.training_len = training_len
        self.forecast_len = forecast_len
        self.feature_len = len(features)
    
    def __len__(self):
        return self.feature_len - self.training_len - self.forecast_len + 1
    
    def __getitem__(self, idx):

        end_trng_idx = idx + self.training_len
        end_target_idx = end_trng_idx + self.forecast_len

        train_features = torch.as_tensor(f_multiplier * self.features[idx:end_trng_idx], dtype=torch.float32)
        # first output to decoder is last input to encoder
        target_features = torch.as_tensor(f_multiplier * self.features[end_trng_idx - 1:end_target_idx - 1], dtype=torch.float32)
        target_values = torch.as_tensor(self.targets[end_trng_idx:end_target_idx], dtype=torch.float32)
 
        return FeaturesAndTarget(train_features, target_features, target_values)

#### Import yfinance

In [ ]:
def download_data(codes, start_date, end_date):
    data = yf.download(codes, start_date, end_date)

    if len(codes) == 1:
        data.columns = [data.columns, codes*len(data.columns)]

    return data.dropna()

In [ ]:
tickers = ["^GSPC", "AAPL", "MSFT", "NKE", "JPM" , "JNJ"]
ticker = "MSFT"
daily_data = download_data(tickers, "2018-05-01", "2022-05-01")
close_data = daily_data['Adj Close']

In [ ]:
prices_data = close_data[ticker].values
prices_data = prices_data.reshape(-1)

plt.plot(prices_data)

In [ ]:
print(prices_data[:10])

In [ ]:
def create_features(prices_data):

    features = pd.DataFrame(dict(px=prices_data)).assign(
        dpx1 = lambda x: x.px.ewm(span=8).mean() - x.px.ewm(span=32).mean(),
        dpx2 = lambda x: x.px.ewm(span=32).mean() - x.px.ewm(span=64).mean(),
        dpx3 = lambda x: x.px.ewm(64).mean() - x.px.ewm(128).mean(),
        dpx4 = lambda x: x.px.ewm(128).mean() - x.px.ewm(256).mean(),
        dpx5 = lambda x: x.px.ewm(12).mean() - x.px.ewm(26).mean(),
        # dpx1 = lambda x: x.px.ewm(20).mean(),
        # dpx2 = lambda x: x.px.ewm(50).mean(),
        # dpx3 = lambda x: x.px.ewm(100).mean(),
        # adj_close_diff_1 = lambda x: x.px.diff(),
        # adj_close_pct_diff = lambda x: x.px.pct_change(),
        # dpx1 = lambda x: np.diff(x.px, prepend=x.px[0]),
        # dpx2 = lambda x: np.diff(x.px, n=2, prepend=[x.px[0], x.px[0]]),
        # dpx4 = lambda x: x.px.ewm(256).mean() - x.px.ewm(512).mean(), 
    )

    return features

In [ ]:
features = create_features(prices_data)

#### Import Data

In [ ]:
FeaturesAndTarget = namedtuple('FeaturesAndTarget', ['train_features', 'target_features', 'target_values'])

#### Splitting Data


In [ ]:
def split_data(features):
    train_idx = int(features.shape[0]*0.6)
    val_idx = int(features.shape[0] * 0.2)
    train_features = features[:train_idx]
    val_features = features[train_idx:train_idx + val_idx]
    test_features = features[train_idx + val_idx:]

    return train_features, val_features, test_features

In [ ]:
train_features, val_features, test_features = split_data(features)
train_features.head()

#### Parameters

In [ ]:
params = dict(
    training_len=60,
    val_len=60,
    max_seq_len=1000,
    batch_size=64,
    d_model=128,
    lr = 1e-3,
    steps_ahead=30, 
    input_dim=1, 
    n_head=8,  
    num_enc_layers=3,
    num_dec_layers=3,
   
    dropout=0.1,
    use_encoder_only=False, 
    use_absolute_enc=True, 
    use_input_projections=True, 
    is_classification=False, 
    device=device, 
    warmup_steps=6000,
    loss='mse',
    scale=True,
    activation="sine",
    k = 60,

    prices_only = True,
    sampling =False,
    threshold=15
)

#### Preprocessing Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train_features_scaler = MinMaxScaler(feature_range=(-1, 1))
train_targets_scaler = MinMaxScaler(feature_range=(0, 1))

In [ ]:
def preprocess_data(train_features, val_features, test_features):

    train_ema = train_features.filter(like='dpx').values
    val_ema = val_features.filter(like='dpx').values
    test_ema = test_features.filter(like='dpx').values

    train_px = train_features['px'].values
    val_px = val_features['px'].values
    test_px = test_features['px'].values
     

    if params['scale']:
        train_features_scaler.fit(train_ema)
        train_targets_scaler.fit(train_px.reshape(-1, 1))

    
    if params['scale']:
        f_train_ema = train_features_scaler.transform(train_ema)
        f_val_ema = train_features_scaler.transform(val_ema)
        f_test_ema = train_features_scaler.transform(test_ema)
        f_train_px = train_targets_scaler.transform(train_px.reshape(-1, 1))
        f_val_px = train_targets_scaler.transform(val_px.reshape(-1, 1))
        f_test_px = train_targets_scaler.transform(test_px.reshape(-1, 1))

        f_train_values = np.concatenate([f_train_px, f_train_ema], axis=1)
        f_val_values = np.concatenate([f_val_px, f_val_ema], axis=1)
        f_test_values = np.concatenate([f_test_px, f_test_ema], axis=1)

    # price only data
    if params["prices_only"]:
        f_train_values = f_train_values[:, 0].reshape(-1, 1)
        f_val_values = f_val_values[:, 0].reshape(-1, 1)
        f_test_values = f_test_values[:, 0].reshape(-1, 1)

        f_train_targets = f_train_values.reshape(-1)
        f_val_targets = f_val_values.reshape(-1)
        f_test_targets = f_test_values.reshape(-1) 
    
    else:
        # all values, incl price

        f_train_targets = f_train_values[:, 0].reshape(-1)
        f_val_targets = f_val_values[:, 0].reshape(-1)
        f_test_targets = f_test_values[:, 0].reshape(-1)
    

    return f_train_values, f_val_values, f_test_values, f_train_targets, f_val_targets, f_test_targets

In [ ]:
f_train_values, f_val_values, f_test_values, f_train_targets, f_val_targets, f_test_targets = preprocess_data(train_features, val_features, test_features)

#### Defining datasets

In [ ]:
training_len = params['training_len']
forecast_len = params['steps_ahead']

print(training_len)
print(forecast_len)

In [ ]:
import random
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

gen = torch.Generator()
gen.manual_seed(1233)

In [ ]:
batch_size = 64

train_dataset = FeaturesDataset(f_train_values, f_train_targets, params['training_len'], forecast_len)
val_dataset = FeaturesDataset(f_val_values, f_val_targets, params['val_len'], forecast_len)
test_dataset = FeaturesDataset(f_test_values, f_test_targets, params['val_len'], forecast_len)

train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=8, worker_init_fn=seed_worker, generator=gen)
val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=8)
train_loader_unshuffled = DataLoader(train_dataset, batch_size, shuffle=False, num_workers=8)

In [ ]:
src, trg_in, trg_out = next(iter(train_loader))
print(src.shape)
print(trg_in.shape)
print(trg_out.shape)

#### Define trainer

In [ ]:
log_dir = "transformer_n_step_ahead_logs"
model_dir = "transformer_n_step_ahead_models"

device = ("cuda:0" if torch.cuda.is_available else "cpu")



def train(ticker, version_name='', model_name ='', ckpt_dir='w2v'):

    if version_name == '':
        version_name = ticker

    if model_name == '':
        model_name = ticker 

    logger = TensorBoardLogger(
        save_dir=log_dir, 
        version=f'{version_name}_{ckpt_dir}'
    )

    checkpoint_callback = ModelCheckpoint(
        monitor="valid_loss_epoch",
        mode="min",
        dirpath=f"{model_dir}/{ckpt_dir}/{model_name}",
        filename="{epoch}-{valid_loss_epoch:.4f}",
        save_last= True,
        save_top_k=2
    )

    early_stopping_callback = EarlyStopping(
        monitor="valid_loss_epoch",
        mode="min",
        patience=100
    )

    metrics = {"loss": "ptl/val_loss"}

    trainer = pl.Trainer(
        max_epochs=200,
        gpus=1,
        logger=logger,
        callbacks=[checkpoint_callback, early_stopping_callback],
        log_every_n_steps=7,
    )

    stock_model = TransformerTrainer(params)

    trainer.fit(stock_model, train_loader, val_loader)

    return trainer, stock_model


In [ ]:
print(params)

#### Evaluation metrics 

In [ ]:
def get_nth_prediction(preds, n=0):
    return preds[n]

def concat_preds(preds):
    return np.concatenate(preds, axis=0)

def revert_transform(values):
    return train_targets_scaler.inverse_transform(values.reshape(-1, 1)).reshape(-1)

def get_prediction_value(preds):
    return [x.cpu().detach().numpy() for x in preds]

def forecast(stock_model, trainer, ckpt_path, test_loader, nth_pred):
    with torch.no_grad():
        predictions = trainer.predict(dataloaders=test_loader, model=stock_model, ckpt_path=ckpt_path)

    all_predictions = concat_preds(predictions)
    predictions_flattened = np.array(flatten(get_nth_prediction(all_predictions, nth_pred)))
    
    return predictions_flattened

def evaluate(pred, y):
    pred = revert_transform(pred)
    y = revert_transform(y)
    
    rmse = calculate_rmse(y, pred)
    mse = rmse ** 2
    mape = calculate_mape(y, pred)

    r = calc_pearson_coeff(pred, y)

    res = dict(
        rmse = rmse,
        mse =mse,
        mape =mape,
        r = r
    )

    return res 


In [ ]:
def mse_loss(pred, target, reduction='sum'):
    loss = np.sum(np.square(pred - target))
    if reduction == "sum":
        return loss
    else:
        return loss / len(pred)

def calculate_rmse(y_true, y_pred):

    rmse = np.sqrt(np.mean((y_true-y_pred)**2))                   
    return rmse

def calculate_mape(y_true, y_pred): 

    y_pred = np.array(y_pred)
    y_true = np.array(y_true)    
    mape = np.mean(np.abs((y_true-y_pred) / y_true))*100    
    return mape


def calc_pearson_coeff(y_pred, y):

    predicted = y_pred
    targets = y
    vy_pred = predicted - np.mean(predicted)
    vy = targets - np.mean(targets)
    denom = np.sum(vy_pred ** 2) * np.sum(vy ** 2)

    corr = np.sum(vy_pred * vy) / np.sqrt(denom)

    return corr

In [ ]:
savefig = False
img_folder = "output_imgs/transformer_n_step/w2v/prices/"

def gen_fig(preds, eval_res, targets, img_folder, title, i = 0, savefig=False):
    if not os.path.exists(img_folder):
        os.makedirs(img_folder)

    plt.figure(figsize=(8, 6))

    start_idx = training_len + i
    end_idx = training_len + len(preds) + i

    filename = f"{img_folder}{ticker}_results.txt" 
    if savefig:
        if os.path.exists(filename):
            f = open(filename, "a")
            f.write("\n")
        else:
            f = open(filename, "x")

    for metric in eval_res.keys():
        print(f"Metric {metric}: {eval_res[metric]}")
        if savefig:
            f.write(f"Metric {metric}: {eval_res[metric]}\n")

    plt.plot(list(range(start_idx, end_idx)), revert_transform(preds), label="predicted")
    plt.plot(list(range(start_idx, end_idx)), revert_transform(targets), label="target")
    plt.plot(revert_transform(f_test_targets[:training_len]), label="trailing")

    plt.title(f"{title}")
    plt.legend()
    plt.grid()
    
    if savefig:
        plt.savefig(f"{img_folder}{ticker}_all_preds.jpg", bbox_inches="tight")

    plt.show()


#### Running model on all stock data

In [ ]:
shorter_tickers = ["^GSPC", "AAPL", "MSFT", "NKE", "JPM" , "JNJ" ]
# need to download data for btc-usd!

daily_data = download_data(shorter_tickers, "2018-05-02", "2022-05-01")
close_data = daily_data['Adj Close']

In [ ]:
print(params)

In [ ]:
shorter_tickers = ["JNJ", "^GSPC"]

ckpt_dir='w2v_sampling'

for v in range(3):
    for ticker in shorter_tickers:
        prices_data = pd.read_csv(f"data/{ticker}.csv")
        prices_data = prices_data['Adj Close'].values
        
        features = create_features(prices_data)

        train_features, val_features, test_features = split_data(features)
        f_train_values, f_val_values, f_test_values, f_train_targets, f_val_targets, f_test_targets = preprocess_data(train_features, val_features, test_features)
        
        training_len = 60
        forecast_len = 30

        batch_size = 64

        gen = torch.Generator()
        gen.manual_seed(1233)

        train_dataset = FeaturesDataset(f_train_values, f_train_targets, params['training_len'], forecast_len)
        val_dataset = FeaturesDataset(f_val_values, f_val_targets, params['training_len'], forecast_len)
        test_dataset = FeaturesDataset(f_test_values, f_test_targets, params['training_len'], forecast_len)

        train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=8, worker_init_fn=seed_worker, generator=gen)
        val_loader = DataLoader(val_dataset, batch_size, shuffle=False, num_workers=8)
        test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=8)
        train_loader_unshuffled = DataLoader(train_dataset, batch_size, shuffle=False, num_workers=8)
        
        global train_output_img_dir
        
        img_dir = f"train_output_imgs/transformer_n_step/{ckpt_dir}/{ticker}/"
        train_output_img_dir = img_dir
        print(train_output_img_dir)

        trainer, stock_model = train(ticker, version_name=f"{ticker}_{v}", model_name=f"{ticker}_{v }", ckpt_dir=ckpt_dir)



In [ ]:
ticker = "JPM"

prices_data = close_data[ticker].values
# prices_data = pd.read_csv(f"data/{ticker}.csv")
# prices_data = prices_data['Adj Close'].values
features = create_features( prices_data = close_data[ticker].values)
train_features, val_features, test_features = split_data(features)
f_train_values, f_val_values, f_test_values, f_train_targets, f_val_targets, f_test_targets = preprocess_data(train_features, val_features, test_features)
test_dataset = FeaturesDataset(f_test_values, f_test_targets, params['val_len'], forecast_len)
test_loader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=8)

In [ ]:
logger = TensorBoardLogger(
    save_dir=log_dir,
    ### TODO: change version when reruun
    version=f'test'
)

checkpoint_callback = ModelCheckpoint(
    monitor="valid_loss_epoch",
    mode="min",
    ### TODO: change dir path when rerun
    dirpath=f"test",
    filename="{epoch}-{valid_loss_epoch:.4f}",
    save_last= True,
    save_top_k=2
)

early_stopping_callback = EarlyStopping(
    monitor="valid_loss_epoch",
    mode="min",
    patience=100
)

metrics = {"loss": "ptl/val_loss"}

trainer = pl.Trainer(
    max_epochs=200,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping_callback],
    log_every_n_steps=7,
)

stock_model = TransformerTrainer(params)

In [ ]:

savefig=False
ckpt_dir = 'w2v_sampling'
stock_model = TransformerTrainer(params)
ckpt_path = 'transformer_n_step_ahead_models/w2v_sampling/final_models/w2v_sampling_jpm.ckpt'
all_preds = forecast(stock_model, trainer, ckpt_path, test_loader, 0)

targets = f_test_targets[training_len:training_len+forecast_len] 
eval_res = evaluate(all_preds, targets)

img_folder = f"output_imgs/transformer_n_step/{ckpt_dir}/{ticker}_1/"
fig_title = f"30-step ahead predictions with encoder-decoder architecture for {ticker}"
gen_fig(all_preds, eval_res, targets, img_folder, fig_title, savefig=savefig)